In [ ]:
!pip install transformers
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# MyDrive 마운트

In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Config Class 정의
from dataclasses import dataclass, field
from typing import List, Optional
import os

@dataclass
class Config:
    pretrained_model_name: str = field(
        default="kogpt2",
    )
    downstream_corpus_dir: str = field(
        default= "/content/drive/MyDrive/11_recipe_generator/data",
    )
    downstream_model_dir: str = field(
        default= "/content/drive/MyDrive/11_recipe_generator/model"
    )
    max_seq_length: int = field(
        default=150,
    )
    save_top_k: int = field(
        default=1,
        metadata={"help": "save top k model checkpoints."}
    )
    monitor: str = field(
        default="min val_loss",
        metadata={"help": "monitor condition (save top k)"}
    )
    seed: int = field(
        default=None,
        metadata={"help": "random seed."}
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    force_download: bool = field(
        default=False,
        metadata={"help": "force to download downstream data and pretrained models."}
    )
    test_mode: bool = field(
        default=False,
        metadata={"help": "Test Mode enables `fast_dev_run`"}
    )
    learning_rate: float = field(
        default=5e-5,
        metadata={"help": "learning rate"}
    )
    epochs: int = field(
        default=3,
        metadata={"help": "max epochs"}
    )
    batch_size: int = field(
        default=8,
        metadata={"help": "batch size. if 0, Let PyTorch Lightening find the best batch size"}
    )
    cpu_workers: int = field(
        default=os.cpu_count(),
        metadata={"help": "number of CPU workers"}
    )
    fp16: bool = field(
        default=False,
        metadata={"help": "Enable train on FP16"}
    )
    tpu_cores: int = field(
        default=0,
        metadata={"help": "Enable TPU with 1 core or 8 cores"}
    )


# Config
args = Config(
    pretrained_model_name = "skt/kogpt2-base-v2",
    downstream_corpus_dir = "/content/drive/MyDrive/11_recipe_generator/data",
    downstream_model_dir = "/content/drive/MyDrive/11_recipe_generator/model",
    max_seq_length = 150,
    batch_size = 4 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    epochs = 10,
    seed = 216,
)


# GenerationExample 자료형
@dataclass
class GenerationExample:
  text: str

# GenerationFeatures 자료형
@dataclass
class GenerationFeatures:
    input_ids: List[int]
    attention_mask: Optional[List[int]] = None
    token_type_ids: Optional[List[int]] = None
    labels: Optional[List[int]] = None

In [ ]:
import logging

def set_logger():
  logger = logging.getLogger("recipe") # 로거 생성
  logger.setLevel(logging.INFO) # 레벨 설정

  if torch.cuda.is_available():
    stream_handler = logging.StreamHandler() # 핸들러 설정
    formatter = logging.Formatter("%(levelname)s: %(name)s - %(message)s") # 출력 포매팅 설정
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

  logger.info("Config %s", args)

  return logger

logger = set_logger()

INFO: recipe - Config Config(pretrained_model_name='skt/kogpt2-base-v2', downstream_corpus_dir='/content/drive/MyDrive/11_recipe_generator/data', downstream_model_dir='/content/drive/MyDrive/11_recipe_generator/model', max_seq_length=150, save_top_k=1, monitor='min val_loss', seed=216, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=10, batch_size=4, cpu_workers=4, fp16=False, tpu_cores=0)


# 라이브러리 로드

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

# 카카오브레인-KoGPT의 토크나이저 

In [ ]:
# unused0 ~ unused1 : 요리 명 구분 / unused2 ~ unused3 : 재료들 구분 / unused4 ~ unused5 : 레시피 구분
tokens_list = ['<unused0>','<unused1>','<unused2>','<unused3>','<unused4>','<unused5>']

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]' , additional_special_tokens = tokens_list
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 카카오브레인-KoGPT 모델 로드

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

# 모델 학습

### 학습 데이터 구축

학습 데이터 파일 train.txt → Corpus Class

In [ ]:
class RecipeCorpus():
  
  def __init__(self):
    pass
  
  def get_corpus(self, data_path, mode):
    PATH = os.path.join(data_path, f"{mode}.txt")
    lines = open(PATH, "r", encoding="utf-8").readlines()

    corpus_lines = []
    for (i, line) in enumerate(lines):
      corpus_lines.append(GenerationExample(text = line))
    return corpus_lines

### CustomDataset

In [ ]:
import time
from typing import List

def _convert_lines_to_features(
        lines: List[GenerationExample],
        tokenizer: AutoTokenizer,
        args: args,
):

    logger.info(
        "tokenize sentences, it could take a lot of time..."
    )
    start = time.time()
    batch_encoding = tokenizer(
        [example.text for example in lines],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    logger.info(
        "tokenize sentences [took %.3f s]", time.time() - start
    )

    features = []
    for i in range(len(lines)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        feature = GenerationFeatures(**inputs, labels=batch_encoding["input_ids"][i])
        features.append(feature)

    for i, example in enumerate(lines[:5]):
        logger.info("*** Example ***")
        logger.info("sentence: %s" % (example.text))
        logger.info("tokens: %s" % (" ".join(tokenizer.convert_ids_to_tokens(features[i].input_ids))))
        logger.info("features: %s" % features[i])

    return features

In [ ]:
from torch.utils.data.dataset import Dataset
from filelock import FileLock

class CustomDataset(Dataset):

  def __init__(self,
               corpus,
               args: args,
               tokenizer : AutoTokenizer,
               mode: Optional[str] = "train",
               convert_lines_to_features_fn = _convert_lines_to_features):
    
    if corpus is not None:
      self.corpus = corpus
    else:
      raise KeyError("fine tuning할 corpus를 입력해주세요")
    
    if not mode in ["train", "val", "test"]:
      raise KeyError(f"{mode}는 사용할 수 없습니다.")
    
    # cached file name
    cached_features_file = os.path.join(
        args.downstream_corpus_dir,
        "cached_{}_{}_{}".format(
            mode,
            tokenizer.__class__.__name__,
            str(args.max_seq_length),
        )
    )

    # cached file이 존재하면 사용, 아니면 생성
    lock_path = cached_features_file + ".lock"
    with FileLock(lock_path):
      if os.path.exists(cached_features_file):
        start = time.time()
        self.features = torch.load(cached_features_file)
        logger.info(f"Loading features from cached file {cached_features_file} [⏰ %.3f s]", time.time() - start)
      
      else:
        corpus_path = args.downstream_corpus_dir
        logger.info(f"Creating features from dataset file at {corpus_path}")

        lines = self.corpus.get_corpus(corpus_path, mode)
        tokenizer.pad_token = tokenizer.eos_token
        self.features = convert_lines_to_features_fn(lines, tokenizer, args)

        start = time.time()
        logger.info("cached file 형태로 저장 중...")
        torch.save(self.features, cached_features_file)
        logger.info("저장 완료: %s [⏰ %.3f s]", cached_features_file, time.time() - start)
  
  def __len__(self):
    return len(self.features)
  
  def __getitem__(self, i):
    return self.features[i]

In [ ]:
corpus = RecipeCorpus()

train_dataset = CustomDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = "train"
)

val_dataset = CustomDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = "val"
)

INFO: recipe - Loading features from cached file /content/drive/MyDrive/11_recipe_generator/data/cached_train_PreTrainedTokenizerFast_150 [⏰ 1.679 s]
INFO: recipe - Loading features from cached file /content/drive/MyDrive/11_recipe_generator/data/cached_val_PreTrainedTokenizerFast_150 [⏰ 0.173 s]


In [ ]:
def data_collator(features):
    """
    Very simple data collator that:
    - simply collates batches of dict-like objects
    - Performs special handling for potential keys named:
        - `label`: handles a single value (int or float) per object
        - `label_ids`: handles a list of values per object
    - does not do any additional preprocessing

    i.e., Property names of the input object will be used as corresponding inputs to the model.
    See glue and ner for example of how it's useful.
    """

    # In this function we'll make the assumption that all `features` in the batch
    # have the same attributes.
    # So we will look at the first element as a proxy for what attributes exist
    # on the whole batch.
    if not isinstance(features[0], dict):
        features = [vars(f) for f in features]

    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    # (it should be automatically the case, but let's make sure of it.)
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    for k, v in first.items():
        if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
            if isinstance(v, torch.Tensor):
                batch[k] = torch.stack([f[k] for f in features])
            else:
                batch[k] = torch.tensor([f[k] for f in features], dtype=torch.long)

    return batch

### DataLoader

In [ ]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

train_dataloader = DataLoader(
    train_dataset,
    batch_size = 4,
    sampler = RandomSampler(train_dataset, replacement=False),
    collate_fn = data_collator,
    drop_last = False,
    num_workers=2
)

In [ ]:
val_dataloader = DataLoader(
    val_dataset,
    batch_size = 4,
    sampler = SequentialSampler(val_dataset),
    collate_fn = data_collator,
    drop_last = False,
    num_workers=2
)

# Fine-tuning

In [ ]:
from transformers import PreTrainedModel
from transformers.optimization import AdamW
from pytorch_lightning import LightningModule
from torch.optim.lr_scheduler import ExponentialLR


class GenerationTask(LightningModule):

    def __init__(self,
                 model: PreTrainedModel,
                 args: args,
    ):
        super().__init__()
        self.model = model
        self.args = args

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=5e-5)
        scheduler = ExponentialLR(optimizer, gamma=0.9)
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def training_step(self, inputs, batch_idx):
        # outputs: CausalLMOutputWithCrossAttentions
        outputs = self.model(**inputs)
        self.log("loss", outputs.loss, prog_bar=False, logger=True, on_step=True, on_epoch=False)
        return outputs.loss

    def validation_step(self, inputs, batch_idx):
        # outputs: CausalLMOutputWithCrossAttentions
        outputs = self.model(**inputs)
        self.log("val_loss", outputs.loss, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return outputs.loss

In [ ]:
task = GenerationTask(model, args)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

def get_trainer(args, return_trainer_only=True):
    ckpt_path = os.path.abspath(args.downstream_model_dir)
    os.makedirs(ckpt_path, exist_ok=True)
    checkpoint_callback = ModelCheckpoint(
        dirpath=ckpt_path,
        save_top_k=args.save_top_k,
        monitor=args.monitor.split()[1],
        mode=args.monitor.split()[0],
        filename='{epoch}-{val_loss:.2f}',
    )
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        callbacks=[checkpoint_callback],
        default_root_dir=ckpt_path,
        # For GPU Setup
        deterministic=torch.cuda.is_available() and args.seed is not None,
        gpus=torch.cuda.device_count() if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32,
        # For TPU Setup
        tpu_cores=args.tpu_cores if args.tpu_cores else None,
    )
    if return_trainer_only:
        return trainer
    else:
        return checkpoint_callback, trainer

In [ ]:
trainer = get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/drive/MyDrive/11_recipe_generator/model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/optimizer.py:381: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  f"Found unsupported keys in the optimizer configuration: {set(extra_keys)}", category=RuntimeWarning

  | Name  | Type            | Params
------------------------------------------
0 | model | GPTJForCausa

Training: 0it [00:00, ?it/s]

RuntimeError: ignored

# 인퍼런스

같은 컨텍스트를 입력했을 때 같은 문장이 나오게 하려면 `그리디 서치`나 `빔 서치`방식 <br/>
매번 다르게 문장이 나오게 하려면 `샘플링`방식 <br/>

* do_sample : 샘플링 방식
* min_length, max_length : 문장 길이 설정
* repetition_penalty : 1.0 보다 커야 페널티가 적용, 1.0은 페널티 적용 X
* no_repetition_ngram_size : n개 이상의 토큰이 반복될 경우 n번째 토큰의 확률을 0으로 만듬
* temperatrue : 0에 가까울 수록 원래 컸던 확률은 더 커지고 원래 작았던 확률은 더 작아짐
* top_k : 가장 확률값이 높은 k개 토큰
* top_p : 누적 확률값이 p이상인 최소 개수의 토큰들을 후보

In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
def input_format(title, ingredients):
  """ transform user's input to fit model's input form
      Args:
        title: 요리 이름
        ingredients: 재료들 => list
      
      return:
        <unused0>title<unused1><unused2>재료1$재료2$재료3<unused3>
  """

  temp1 = '<unused0>' + title + '<unused1>'
  temp2 = '<unused2>'
  for ingredient in ingredients:
    temp2 += ingredient + '$'
  temp2 += '<unused3>'
  return temp1 + temp2


def inference(
    prompt, min_length=50, max_length=500, top_p=1.0, top_k=50, repetition_penalty=1.5, no_repeat_ngram_size=0, temperature=1.0):
  try:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
      generated_ids = model.generate(
          input_ids,
          do_sample=True,
          top_p=float(top_p),
          top_k=int(top_k),
          min_length = int(min_length),
          max_length=int(max_length),
          repetition_penalty = float(repetition_penalty),
          no_repeat_ngram_size = int(no_repeat_ngram_size),
          temperature = float(temperature),
          pad_token_id=tokenizer.pad_token_id,
          eos_token_id=tokenizer.eos_token_id,
          bos_token_id=tokenizer.bos_token_id,
          use_cache=True
      )
      generated_sentence = tokenizer.decode(generated_ids[0])
  except:
    generated_sentence = "오류"

  return {
      'result': generated_sentence
  }

In [ ]:
sentence = input_format("비빔밥", ["콩나물", "밥", "고추장"])
inference(sentence)

{'result': '<unused0> 비빔밥<unused1><unused2> 콩나물$밥$고추장$<unused3><unused4> 끓는 물에 데친콩나물을 찬물에 헹궈 물기를 꼭 짠 뒤 잘게 썬다. 그릇에 밥을 담고 고추장과 송송 다진 파를 얹어 낸 후 불에서 내린다.<unused5> \n</s>'}

# 모델 저장

In [ ]:
model_save_name = '20220606'
model_file='.pt'
path = f"./{model_save_name}{model_file}" 

In [ ]:
torch.save(model.state_dict(), path)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loaded_model = model.load_state_dict(torch.load("/content/drive/MyDrive/11_recipe_generator/model/20220606.pt"))